# 通过XPATH解析HTML

XPath在Python的爬虫学习中，起着举足轻重的地位，对比正则表达式 re两者可以完成同样的工作，实现的功能也差不多，但XPath明显比re具有优势，在网页分析上使re退居二线。<br>
XPath全称为XML Path Language，是一种小型的查询语言 <br>
1） 可在XML中查找信息 <br>
2） 支持HTML的查找 <br>
3） 通过元素和属性进行导航 <br>


## Python开发使用XPath的条件
由于XPath属于lxml库模块，所以首先要安装库lxml

## XPath的简单调用方法

首先加载lxml

In [3]:
from lxml import etree
import re

定义方法，从html文件读取html源码，并通过etree加载。<br>
关键部分在于：<br>
* 编码部分，最好显式声明为utf-8
* etree.HTML(html源码)，这句代码的目的是将源码转化为能被XPath匹配的格式

In [4]:
def gethtmlrootnodeforxpath(file):
    with open(file, 'r', encoding='utf-8', errors='ignore') as f:
        htmlbytes = bytes(bytearray(f.read(), encoding='utf-8'))
        html = etree.HTML(htmlbytes)
        return html

In [5]:
htmlobject = gethtmlrootnodeforxpath('./html/167711672.htm')

In [6]:
print(htmlobject)

<Element html at 0x56256c8>


<font color='red'>如果有将html文件转为纯文本的需求，以下代码，就可以解决这个问题</font>

In [24]:
content = htmlobject.xpath('string(.)')
contents = content.split('\n')
for text in contents:
    if len(text.strip()) > 0:
        #print(text.strip())
        pass

## XPath的基本使用方法

首先讲一下XPath的基本语法知识:

四中标签的使用方法:<br>
* 1) // 双斜杠 定位根节点，会对全文进行扫描，在文档中选取所有符合条件的内容，以列表的形式返回。 
* 2) / 单斜杠 寻找当前标签路径的下一层路径标签或者对当前路标签内容进行操作 
* 3) /text() 获取当前路径下的文本内容 
* 4) /@xxxx 提取当前路径下标签的属性值 
* 5) | 可选符 使用|可选取若干个路径 如//p | //div 即在当前路径下选取所有符合条件的p标签和div标签。 
* 6) . 点 用来选取当前节点 
* 7) .. 双点 选取当前节点的父节点 
* 另外还有starts-with(@属性名称,属性字符相同部分)，string(.)两种重要的特殊方法后面将重点讲。

### 简单的例子

In [8]:
html = '''
<!DOCTYPE html>
<html>
    <head lang="en">
    <title>测试</title>
    <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
</head>
<body>
    <div id="content">
        <ul id="ul">
            <li>NO.1</li>
            <li>NO.2</li>
            <li>NO.3</li>
        </ul>
        <ul id="ul2">
            <li>one</li>
            <li>two</li>
        </ul>
    </div>
    <div id="url">
        <a href="http:www.58.com" title="58">58</a>
        <a href="http:www.csdn.net" title="CSDN">CSDN</a>
    </div>
</body>
</html>
'''

使用id属性来定位哪个div和ul被匹配，使用text()获取文本内容

In [9]:
selector = etree.HTML(html)
content=selector.xpath('//div[@id="content"]/ul[@id="ul"]/li/text()')
for i in content:
    print(i)

NO.1
NO.2
NO.3


这里使用//从全文中定位符合条件的a标签，使用“@标签属性”获取a便签的href属性值

In [10]:
con = selector.xpath('//a/@href')
for each in con:
    print(each)

http:www.58.com
http:www.csdn.net


使用绝对路径定位

In [11]:
con=selector.xpath('/html/body/div/a/@title') 
print(len(con))
for each in con:
    print(each)

2
58
CSDN


## 进阶用法

* starts-with 解决标签属性值以相同字符串开头的情况

In [12]:
html="""
    <body>
        <div id="aa">aa</div>
        <div id="ab">ab</div>
        <div id="ac">ac</div>
    </body>
    """
selector=etree.HTML(html)
#这里使用starts-with方法提取div的id标签属性值开头为a的div标签
content=selector.xpath('//div[starts-with(@id,"a")]/text()') 
for each in content:
    print(each)

aa
ab
ac


*  string(.) 标签套标签，具体含义就是采集xpath路径所包含的所有标签内的文本

In [13]:
html="""
<div id="a">
left
<div id="b">
right
<ul>
up
<li>down</li>
</ul>
east
</div>
west
</div>
"""
#下面是没有用string方法的输出
sel=etree.HTML(html)
con=sel.xpath('//div[@id="a"]/text()')
for i in con:
    print(i)   #输出内容为left west

data=sel.xpath('//div[@id="a"]')[0]
info=data.xpath('string(.)')
#输出为 全部内容
print(info)


left


west


left

right

up
down

east

west



## 获得XPath的方法

* 使用Chrome浏览器
* 在具体要获取的元素，鼠标右键单击
* 选择Inspect，定位HTML标签

![chrome](./image/2018-09-08 18_52_55-PL SVUL Protector 2018 Combined Document.png)

* 定位到标签之后，鼠标右键源码，copy -> copy xpath

![chrome](./image/2018-09-08 18_55_35-PL SVUL Protector 2018 Combined Document.png)

于是我们就得到具体的XPath：/html/body/div[127]/div/table/tbody/tr[6]/td[1]/div/font<br>
真实情况中，我们不会这样写绝对定位路径，往往会通过/html/body/div/div/table的方式，尝试获得Table列表，然后根据Table中的表头信息，去定位我们想要的table

## 实战

XPath在实际工作中使用的场景:<br>
基金公司公布的Share Name往往集中在某些Table，DIV内，使用XPath能够很方便定位

### Document Id: 167711672

需求：获取文档中的Share Name。<br>
Share Name位于文章的表格中，且为第一列
![table](./image/2018-08-15 18_26_33-PL SVUL Protector 2018 Combined Document.png)

将html对象加载到lxml中：

In [14]:
htmlobject = gethtmlrootnodeforxpath('./html/167711672.htm')

经过分析，我们如果需要得到table，可以通过如下路径：/html/body/div/div/table,得到一堆table

In [15]:
tablelist = htmlobject.xpath('/html/body/div/div/table')
print(len(tablelist))

74


如何找到我想要的table呢？<br>
* 定位表头<br>
经过分析，该table的第二列的文字是investment objective summary，这其实可以区分其他非share name描述的table，然后我们可以获取任意该列的xpath<br>
/html/body/div[127]/div/table/tbody/tr[4]/td[2]/div/font<br>
我们忽略tbody之前的内容，可以发现能够通过tbody/tr[4]/td[2]进行定位了。
* 定位share name<br>
我们定位任意share name: /html/body/div[127]/div/table/tbody/tr[6]/td[1]/div/font<br>
因为share name有很多，所以我们通过/tbody/tr/td[1]进行过滤，获得列表即可
* 加上限制<br>
因为表格中有跨列的内容，如ADVANCED SERIES TRUST，所以还需要加入span相关的排除描述:<br>
tbody/tr/td[not(@colspan)][1]

In [16]:
def getpureelementtext(element):
    text = element.xpath('string(.)').strip()
    text = text.replace('\n', ' ')
    text = re.sub('( ){2,}', ' ', text)
    return text

In [17]:
def searchbykeyword(keyword, text):
    pattern = re.compile(keyword)
    return re.search(pattern, text)

In [23]:
keyword = r'(\b([0-9A-Zi])\S*( )\b){2,}'
sharecount = 1
for table in tablelist:
    header = table.xpath('tbody/tr[4]/td[2]')
    if len(header) > 0:
        temp = getpureelementtext(header[0])
#         print(temp)
        # 通过header文字，得到正确的table
        if temp == 'Investment Objective Summary':
            # 通过share name的xpath，获得相应的element list
            sharelist = table.xpath('tbody/tr/td[not(@colspan)][1]')
            for index, share in enumerate(sharelist):
                sharename = getpureelementtext(share)
                # Share Name一般都是大写字母开头的单词组合而成
                if searchbykeyword(keyword, sharename) is not None:
                    print('share {0}, name is: {1}'.format(sharecount, getpureelementtext(share)))
                    sharecount += 1

share 1, name is: AST Advanced Strategies Portfolio
share 2, name is: AST Balanced Asset Allocation Portfolio
share 3, name is: AST BlackRock Global Strategies Portfolio
share 4, name is: AST BlackRock Low Duration Bond Portfolio
share 5, name is: AST BlackRock/Loomis Sayles Bond Portfolio
share 6, name is: AST Fidelity Institutional AMSM Quantitative Portfolio (formerly AST FI Pyramis® Quantitative Portfolio)
share 7, name is: AST Goldman Sachs Mid-Cap Growth Portfolio
share 8, name is: AST Hotchkis & Wiley Large-Cap Value Portfolio
share 9, name is: AST International Value Portfolio
share 10, name is: AST J.P. Morgan International Equity Portfolio
share 11, name is: AST J.P. Morgan Strategic Opportunities Portfolio
share 12, name is: AST Loomis Sayles Large-Cap Growth Portfolio
share 13, name is: AST MFS Global Equity Portfolio
share 14, name is: AST MFS Growth Portfolio
share 15, name is: AST Preservation Asset Allocation Portfolio
share 16, name is: AST Prudential Growth Allocation

### Document Id: 169431423

需求：获取文档中的Share Name。<br>
Share Name位于文章的DIV中<br>
![document](./image/2018-08-23 10_25_51-BOA CVUL Future.png)

将html对象加载到lxml中：

In [19]:
htmlfordivobject = gethtmlrootnodeforxpath('./html/169431423.htm')

经过分析，发现share name都是在div段落中，并且有特定的属性和样式

![documents](./image/2018-09-08 19_43_32-BOA CVUL Future.png)

* 首先定位div段落: /html/body/div[206]/div[1]，结合属性，我们可以得到这样的xpath: /html/body/div/div[@type=\"Block\"]
* 然后定位share name，结合样式，我们可以得到这样的xpath: div[contains(@style,\"FONT-SIZE: 9pt;\") and contains(@style, \"FONT-WEIGHT: bold;\")]

具体代码如下：

In [21]:
keyword = r'(\b([0-9A-Zi])\S*( )\b){2,}'
sharecount = 1
for block in htmlfordivobject.xpath('/html/body/div/div[@type=\"Block\"]'):
    sharelist = block.xpath('div[contains(@style,\"FONT-SIZE: 9pt;\") and contains(@style, \"FONT-WEIGHT: bold;\")]')
    for share in sharelist:
        sharename = getpureelementtext(share)
        # Share Name一般都是大写字母开头的单词组合而成
        if searchbykeyword(keyword, sharename) is not None:
            print('share {0}, name is: {1}'.format(sharecount, getpureelementtext(share)))
            sharecount += 1
        

share 1, name is: AllianceBernstein Variable Products Series Fund, Inc. - AB VPS Growth and Income Portfolio: Class A
share 2, name is: AllianceBernstein Variable Products Series Fund, Inc. - AB VPS International Value Portfolio: Class A
share 3, name is: AllianceBernstein Variable Products Series Fund, Inc. - AB VPS Small/Mid Cap Value Portfolio: Class A
share 4, name is: American Century Variable Portfolios II, Inc. - American Century VP Inflation Protection Fund: Class I
share 5, name is: American Century Variable Portfolios, Inc. - American Century VP Capital Appreciation Fund: Class I
share 6, name is: American Century Variable Portfolios, Inc. - American Century VP Income & Growth Fund: Class I
share 7, name is: American Century Variable Portfolios, Inc. - American Century VP International Fund: Class I
share 8, name is: American Century Variable Portfolios, Inc. - American Century VP Mid Cap Value Fund: Class I
share 9, name is: American Century Variable Portfolios, Inc. - Ameri

## 不太适合用XPath的场景

刚刚的例子是针对从具体的某一段HTML标签拿取相关信息。<br>
但是如果需要从文本中，根据某些关键字，如new, close定位相应的段落，即确定action；<br>并且再从上下文获取具体的share name或者pending date<br>
根据实际经验，感觉通过转为纯文本，然后正则定位或许更合适。<br>
用XPath可能反而会让事情复杂化，而且<b>XPath的有效性，取决于html的样式不能频繁发生变化。</b>